<a href="https://colab.research.google.com/github/sifiphd/IntroStatLearn/blob/master/BERT_CIKM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [3]:
pip install transformers

In [4]:
import torch

# Verifica se CUDA (GPU) è disponibile
print("CUDA Available:", torch.cuda.is_available())

# Stampa il nome della GPU, se presente
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("Using CPU")

CUDA Available: True
GPU Name: Tesla T4


ho verificato che non ho CUDA disponibile, questo significa che lavoro sulla CPU locale. Questo potrebbe rallentare l'addestramento del modello BERT. Nel caso in cui l'addestramento dovesse richiedere troppo tempo, allora potrei sia usare una macchina virtuale e quindi per esempio usare google colab sfruttando la GPU, e sia usare un modello più 'leggero', come DistilBERT o ALBERT)

In [5]:
from torch.utils.data import DataLoader, TensorDataset


In [6]:
import transformers
print(transformers.__version__)


4.51.3


In [7]:
from torch.optim import AdamW


In [8]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [10]:
# 1. Caricamento dei dati dal CSV
df_w= pd.read_csv('/content/CMT_matrix_cikm.csv', delimiter=',')


In [11]:
df_w

,Unnamed: 0,case_id,act1,act2,act3,act4,act5,act6,act7
0,0,Application_1219772874pp1,A_Create Application,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created
1,1,Application_1219772874pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,O_Sent (mail and online)
2,2,Application_1219772874pp3,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,O_Sent (mail and online),W_Complete application
3,3,Application_2037628374pp1,A_Create Application,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created
4,4,Application_2037628374pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,O_Sent (mail and online)
...,...,...,...,...,...,...,...,...,...
94522,94522,Application_523628252pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Cancelled
94523,94523,Application_523628252pp3,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Cancelled,O_Cancelled
94524,94524,Application_961957338pp1,A_Create Application,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created
94525,94525,Application_961957338pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Cancelled


In [12]:
df_w.columns

Index(['Unnamed: 0', 'case_id', 'act1', 'act2', 'act3', 'act4', 'act5', 'act6',
       'act7'],
      dtype='object')

In [13]:
df_w=df_w[['case_id', 'act1', 'act2', 'act3', 'act4', 'act5', 'act6',
       'act7']]

In [14]:
df_w

,case_id,act1,act2,act3,act4,act5,act6,act7
0,Application_1219772874pp1,A_Create Application,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created
1,Application_1219772874pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,O_Sent (mail and online)
2,Application_1219772874pp3,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,O_Sent (mail and online),W_Complete application
3,Application_2037628374pp1,A_Create Application,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created
4,Application_2037628374pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,O_Sent (mail and online)
...,...,...,...,...,...,...,...,...
94522,Application_523628252pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Cancelled
94523,Application_523628252pp3,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Cancelled,O_Cancelled
94524,Application_961957338pp1,A_Create Application,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created
94525,Application_961957338pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Cancelled


il df precedente è il df con sliding windows, che contiene solo la sequenza di attività senza l'ultima


In [15]:
# Concatenazione delle attività per creare una sequenza
df_w['activities'] = df_w[['act1', 'act2', 'act3', 'act4','act5', 'act6', 'act7']].astype(str).agg(' '.join, axis=1)


In [16]:
df_w

,case_id,act1,act2,act3,act4,act5,act6,act7,activities
0,Application_1219772874pp1,A_Create Application,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Create Application W_Complete application W_...
1,Application_1219772874pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,O_Sent (mail and online),W_Complete application W_Complete application ...
2,Application_1219772874pp3,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,O_Sent (mail and online),W_Complete application,W_Complete application A_Concept A_Accepted O_...
3,Application_2037628374pp1,A_Create Application,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Create Application W_Complete application W_...
4,Application_2037628374pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,O_Sent (mail and online),W_Complete application W_Complete application ...
...,...,...,...,...,...,...,...,...,...
94522,Application_523628252pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Cancelled,W_Complete application W_Complete application ...
94523,Application_523628252pp3,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Cancelled,O_Cancelled,W_Complete application A_Concept A_Accepted O_...
94524,Application_961957338pp1,A_Create Application,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Create Application W_Complete application W_...
94525,Application_961957338pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Cancelled,W_Complete application W_Complete application ...


In [17]:
df_last= pd.read_csv('/content/SVE.csv', delimiter=',')


In [18]:
df_last

,Unnamed: 0,activity_10
0,0,W_Call after offers
1,1,W_Call after offers
2,2,W_Complete application
3,3,O_Cancelled
4,4,W_Complete application
...,...,...
31504,31504,W_Complete application
31505,31505,W_Complete application
31506,31506,W_Complete application
31507,31507,W_Complete application


In [19]:
df_last=df_last[['activity_10']]

In [20]:
df_last

,activity_10
0,W_Call after offers
1,W_Call after offers
2,W_Complete application
3,O_Cancelled
4,W_Complete application
...,...
31504,W_Complete application
31505,W_Complete application
31506,W_Complete application
31507,W_Complete application


In [21]:
# Supponiamo che la colonna da ripetere sia 'activity'
col = df_last['activity_10'].reset_index(drop=True)  # Series con 31.509 righe

# Ripeti ogni elemento 3 volte
repeated_col = col.repeat(3).reset_index(drop=True)  # → 94.527 righe


In [22]:
repeated_col

,activity_10
0,W_Call after offers
1,W_Call after offers
2,W_Call after offers
3,W_Call after offers
4,W_Call after offers
...,...
94522,W_Complete application
94523,W_Complete application
94524,W_Complete application
94525,W_Complete application


In [23]:
df_w['last_activity']=repeated_col

In [24]:
df_w

,case_id,act1,act2,act3,act4,act5,act6,act7,activities,last_activity
0,Application_1219772874pp1,A_Create Application,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Create Application W_Complete application W_...,W_Call after offers
1,Application_1219772874pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,O_Sent (mail and online),W_Complete application W_Complete application ...,W_Call after offers
2,Application_1219772874pp3,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,O_Sent (mail and online),W_Complete application,W_Complete application A_Concept A_Accepted O_...,W_Call after offers
3,Application_2037628374pp1,A_Create Application,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Create Application W_Complete application W_...,W_Call after offers
4,Application_2037628374pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,O_Sent (mail and online),W_Complete application W_Complete application ...,W_Call after offers
...,...,...,...,...,...,...,...,...,...,...
94522,Application_523628252pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Cancelled,W_Complete application W_Complete application ...,W_Complete application
94523,Application_523628252pp3,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Cancelled,O_Cancelled,W_Complete application A_Concept A_Accepted O_...,W_Complete application
94524,Application_961957338pp1,A_Create Application,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Create Application W_Complete application W_...,W_Complete application
94525,Application_961957338pp2,W_Complete application,W_Complete application,A_Concept,A_Accepted,O_Create Offer,O_Created,A_Cancelled,W_Complete application W_Complete application ...,W_Complete application


In [25]:
df_bert= df_w[['activities','last_activity']].copy()

In [26]:
df_bert

,activities,last_activity
0,A_Create Application W_Complete application W_...,W_Call after offers
1,W_Complete application W_Complete application ...,W_Call after offers
2,W_Complete application A_Concept A_Accepted O_...,W_Call after offers
3,A_Create Application W_Complete application W_...,W_Call after offers
4,W_Complete application W_Complete application ...,W_Call after offers
...,...,...
94522,W_Complete application W_Complete application ...,W_Complete application
94523,W_Complete application A_Concept A_Accepted O_...,W_Complete application
94524,A_Create Application W_Complete application W_...,W_Complete application
94525,W_Complete application W_Complete application ...,W_Complete application


In [27]:
# 2. Divisione dei dati in train e test (stratificato)
train_df, test_df = train_test_split(df_bert, test_size=0.2, random_state=42, stratify=df_bert['last_activity'])

In [28]:
train_df

,activities,last_activity
40811,W_Complete application A_Concept A_Accepted O_...,W_Call after offers
65958,A_Create Application A_Concept W_Complete appl...,O_Created
92392,W_Complete application W_Complete application ...,W_Call after offers
21595,A_Submitted W_Handle leads W_Handle leads W_Co...,O_Sent (mail and online)
20538,A_Create Application W_Complete application W_...,W_Call after offers
...,...,...
58629,A_Create Application A_Submitted W_Handle lead...,W_Complete application
11405,W_Handle leads W_Handle leads W_Complete appli...,A_Accepted
15457,A_Submitted W_Handle leads W_Handle leads W_Co...,O_Created
91000,W_Complete application W_Complete application ...,W_Call after offers


In [29]:
test_df

,activities,last_activity
36096,A_Create Application A_Concept W_Complete appl...,W_Call after offers
617,W_Complete application A_Concept A_Accepted O_...,W_Call after offers
11147,W_Complete application A_Concept A_Accepted O_...,W_Call after offers
75390,A_Create Application A_Submitted W_Handle lead...,W_Complete application
19204,A_Submitted W_Handle leads W_Handle leads W_Co...,O_Create Offer
...,...,...
23676,A_Create Application A_Concept W_Complete appl...,W_Call after offers
42236,W_Handle leads W_Handle leads W_Handle leads W...,W_Complete application
1469,W_Handle leads W_Handle leads W_Complete appli...,O_Created
20277,A_Create Application A_Submitted W_Handle lead...,W_Complete application


In [30]:
# Get the lists of activities and their labels output.
traces = df_bert.activities.values
label = df_bert.last_activity.values

In [31]:
traces

array(['A_Create Application W_Complete application W_Complete application A_Concept A_Accepted O_Create Offer O_Created',
       'W_Complete application W_Complete application A_Concept A_Accepted O_Create Offer O_Created O_Sent (mail and online)',
       'W_Complete application A_Concept A_Accepted O_Create Offer O_Created O_Sent (mail and online) W_Complete application',
       ...,
       'A_Create Application W_Complete application W_Complete application A_Concept A_Accepted O_Create Offer O_Created',
       'W_Complete application W_Complete application A_Concept A_Accepted O_Create Offer O_Created A_Cancelled',
       'W_Complete application A_Concept A_Accepted O_Create Offer O_Created A_Cancelled O_Cancelled'],
      dtype=object)

In [32]:
label

array(['W_Call after offers', 'W_Call after offers',
       'W_Call after offers', ..., 'W_Complete application',
       'W_Complete application', 'W_Complete application'], dtype=object)

In [33]:
# 3. Tokenizzazione con BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Nella prossima cella faccio solo il print per vedere quali sono gli output delle attività all'inizio, le attività tokenizzate e gli id ottenuti

In [34]:
# original process
print(' Original: ', traces[0])

# Print the process split into tokens.
print('Tokenized: ', tokenizer.tokenize(traces[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(traces[0])))

 Original:  A_Create Application W_Complete application W_Complete application A_Concept A_Accepted O_Create Offer O_Created
Tokenized:  ['a', '_', 'create', 'application', 'w', '_', 'complete', 'application', 'w', '_', 'complete', 'application', 'a', '_', 'concept', 'a', '_', 'accepted', 'o', '_', 'create', 'offer', 'o', '_', 'created']
Token IDs:  [1037, 1035, 3443, 4646, 1059, 1035, 3143, 4646, 1059, 1035, 3143, 4646, 1037, 1035, 4145, 1037, 1035, 3970, 1051, 1035, 3443, 3749, 1051, 1035, 2580]


Nella prossima cella faccio delle liste dove conservo le informazioni ottenute dalle operazioni precedenti

In [35]:
# faccio prima l'encoding, che consiste nella:
# 1) divisione in token
# 2) Aggiunta del token `[CLS]` all'inizio
# 3) Aggiunta del token `[SEP]` alla fine
# 4) mappatura dei token negli IDs.

input_ids = []

# quindi per ogni processo, o meglio per ogni attività nel processo:
for trace in traces:

    encoded_trace = tokenizer.encode(
                        trace
                   )


#aggiungo l'attività codificata alla lista
    input_ids.append(encoded_trace)


# Print sentence 0, now as a list of IDs.
print('Original trace: ', traces[0])
print('Trace in token IDs:', input_ids[0])

Original trace:  A_Create Application W_Complete application W_Complete application A_Concept A_Accepted O_Create Offer O_Created
Trace in token IDs: [101, 1037, 1035, 3443, 4646, 1059, 1035, 3143, 4646, 1059, 1035, 3143, 4646, 1037, 1035, 4145, 1037, 1035, 3970, 1051, 1035, 3443, 3749, 1051, 1035, 2580, 102]


il risultato è diverso dal precedente, perchè sono stati aggiunti i token [CLS] e [SEP] che prima non sono stati aggiunti, perchè mancava la parte di encoding.

In [36]:
trace

'W_Complete application A_Concept A_Accepted O_Create Offer O_Created A_Cancelled O_Cancelled'

In [37]:
print('Max trace length: ', max([len(trace) for trace in input_ids]))

Max trace length:  35


nella tokenizzazione vengono suddivise le attività, cioè A_Create Application viene considerato come:
- 'a', '_', 'create', 'application'
- che sono quindi cinque diversi token

- per questo ottengo come lunghezza massima della traccia 35...perchè mi conta tutti gli id

In [38]:
print('Min trace length: ', min([len(trace) for trace in input_ids]))

Min trace length:  25


In [39]:
pip install keras

In [40]:
pip install tensorflow

In [41]:

from keras.preprocessing.sequence import pad_sequences


MAX_LEN = 35

#Padding the input to the max length that is 35
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long",
                          value=0, truncating="post", padding="post")

In [42]:
print('Min trace length: ', min([len(trace) for trace in input_ids]))

Min trace length:  35


Nelle tracce, anche se erano tutte uguali, a seguito della tokenizzazione (e quindi della divisone in parole etc.) ho ottenuto delle tracce (ovvero dei vettori...) più corte ed altre più lunghe (quindi ho dovuto aggiungere degli 0)

Una volta effettuato il padding ottengo delle tracce tutte con la stessa lunghezza.

Poichè ciò ha richiesto l'utilizzo del padding, allora devo utilizzare l'attention mask, perchè in questo modo riesco a dare attenzione solo alle attività e non al padding

In [43]:
# Creo l'attention mask.
# Se il token ID è 0, allora è padding, quindi metti il masking a 0.
# Se il token ID è > 0, allora c'è un vero token, quindi metti il masking a 1.


attention_masks = []

for trace in input_ids:

    att_mask = [int(token_id > 0) for token_id in trace]

    # Store the attention mask per questa traccia.
    attention_masks.append(att_mask)

In [44]:
attention_masks

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [45]:
label

array(['W_Call after offers', 'W_Call after offers',
       'W_Call after offers', ..., 'W_Complete application',
       'W_Complete application', 'W_Complete application'], dtype=object)

In [46]:
input_ids

array([[ 101, 1037, 1035, ...,    0,    0,    0],
       [ 101, 1059, 1035, ...,    0,    0,    0],
       [ 101, 1059, 1035, ...,    0,    0,    0],
       ...,
       [ 101, 1037, 1035, ...,    0,    0,    0],
       [ 101, 1059, 1035, ...,    0,    0,    0],
       [ 101, 1059, 1035, ...,    0,    0,    0]])

In [47]:
encoded_activities= pd.read_csv('/content/activities.csv', delimiter=',')


In [48]:
encoded_activities

,Activity,Value
0,A_Create Application,0
1,A_Submitted,1
2,W_Handle leads,2
3,W_Complete application,3
4,A_Concept,4
5,A_Accepted,5
6,O_Create Offer,6
7,O_Created,7
8,O_Sent (mail and online),8
9,W_Call after offers,9


In [49]:
# Supponiamo che il tuo DataFrame si chiami df
activity_dict = pd.Series(encoded_activities['Value'].values, index=encoded_activities['Activity']).to_dict()


In [50]:
label = [activity_dict[act] for act in label]


In [51]:
label

[9,
 9,
 9,
 9,
 9,
 9,
 3,
 3,
 3,
 12,
 12,
 12,
 3,
 3,
 3,
 2,
 2,
 2,
 7,
 7,
 7,
 9,
 9,
 9,
 8,
 8,
 8,
 7,
 7,
 7,
 3,
 3,
 3,
 3,
 3,
 3,
 9,
 9,
 9,
 6,
 6,
 6,
 8,
 8,
 8,
 3,
 3,
 3,
 3,
 3,
 3,
 6,
 6,
 6,
 3,
 3,
 3,
 3,
 3,
 3,
 8,
 8,
 8,
 6,
 6,
 6,
 4,
 4,
 4,
 9,
 9,
 9,
 8,
 8,
 8,
 3,
 3,
 3,
 6,
 6,
 6,
 8,
 8,
 8,
 3,
 3,
 3,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 7,
 7,
 7,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 9,
 9,
 9,
 5,
 5,
 5,
 7,
 7,
 7,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 9,
 9,
 9,
 3,
 3,
 3,
 9,
 9,
 9,
 9,
 9,
 9,
 4,
 4,
 4,
 9,
 9,
 9,
 9,
 9,
 9,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 3,
 3,
 3,
 8,
 8,
 8,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 5,
 5,
 5,
 3,
 3,
 3,
 3,
 3,
 3,
 5,
 5,
 5,
 9,
 9,
 9,
 3,
 3,
 3,
 9,
 9,
 9,
 9,
 9,
 9,
 7,
 7,
 7,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 3,
 3,
 3,
 8,
 8,
 8,
 3,
 3,
 3,
 6,
 6,
 6,
 6,
 6,
 6,
 8,
 8,
 8,
 7,
 7,
 7,
 9,
 9,
 9,
 2,
 2,
 2,
 5,
 5,
 5,
 

In [52]:
print("Etichette originali:", label)

Etichette originali: [9, 9, 9, 9, 9, 9, 3, 3, 3, 12, 12, 12, 3, 3, 3, 2, 2, 2, 7, 7, 7, 9, 9, 9, 8, 8, 8, 7, 7, 7, 3, 3, 3, 3, 3, 3, 9, 9, 9, 6, 6, 6, 8, 8, 8, 3, 3, 3, 3, 3, 3, 6, 6, 6, 3, 3, 3, 3, 3, 3, 8, 8, 8, 6, 6, 6, 4, 4, 4, 9, 9, 9, 8, 8, 8, 3, 3, 3, 6, 6, 6, 8, 8, 8, 3, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 7, 7, 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 9, 9, 5, 5, 5, 7, 7, 7, 4, 4, 4, 4, 4, 4, 3, 3, 3, 9, 9, 9, 3, 3, 3, 9, 9, 9, 9, 9, 9, 4, 4, 4, 9, 9, 9, 9, 9, 9, 8, 8, 8, 9, 9, 9, 9, 9, 9, 3, 3, 3, 8, 8, 8, 7, 7, 7, 7, 7, 7, 8, 8, 8, 5, 5, 5, 3, 3, 3, 3, 3, 3, 5, 5, 5, 9, 9, 9, 3, 3, 3, 9, 9, 9, 9, 9, 9, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 3, 3, 3, 8, 8, 8, 3, 3, 3, 6, 6, 6, 6, 6, 6, 8, 8, 8, 7, 7, 7, 9, 9, 9, 2, 2, 2, 5, 5, 5, 3, 3, 3, 5, 5, 5, 3, 3, 3, 9, 9, 9, 3, 3, 3, 6, 6, 6, 7, 7, 7, 2, 2, 2, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 3, 3, 3, 4, 4, 4, 7, 7, 7, 4, 4, 4, 5, 5, 5, 9, 9, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 7, 7, 9, 9, 9, 4, 4, 4, 5, 5, 5, 6, 

In [53]:
# Chiamiamo la funzione train_test_split() da sklearn
from sklearn.model_selection import train_test_split

# 80% train, 10% validation, 10% test
train_inputs, temp_inputs, train_labels, temp_labels = train_test_split(
    input_ids, label, random_state=2018, test_size=0.2)

#temp_inputs è una parte di dataset di input temporanea, che mi serve poi ad effettuare l'ulteriore divisione in train e test

validation_inputs, test_inputs, validation_labels, test_labels = train_test_split(
    temp_inputs, temp_labels, random_state=2018, test_size=0.5)

# Stessa cosa per le attention masks
train_masks, temp_masks, _, _ = train_test_split(
    attention_masks, label, random_state=2018, test_size=0.2)

validation_masks, test_masks, _, _ = train_test_split(
    temp_masks, temp_labels, random_state=2018, test_size=0.5)

In [54]:
validation_inputs, test_inputs, validation_labels, test_labels

(array([[ 101, 1037, 1035, ...,    0,    0,    0],
        [ 101, 1059, 1035, ...,    0,    0,    0],
        [ 101, 1059, 1035, ...,    0,    0,    0],
        ...,
        [ 101, 1037, 1035, ...,    0,    0,    0],
        [ 101, 1037, 1035, ...,    0,    0,    0],
        [ 101, 1037, 1035, ...,    0,    0,    0]]),
 array([[ 101, 1037, 1035, ...,    0,    0,    0],
        [ 101, 1059, 1035, ...,    0,    0,    0],
        [ 101, 1037, 1035, ...,    0,    0,    0],
        ...,
        [ 101, 1037, 1035, ...,    0,    0,    0],
        [ 101, 1059, 1035, ...,    0,    0,    0],
        [ 101, 1059, 1035, ...,    0,    0,    0]]),
 [3,
  3,
  8,
  5,
  7,
  3,
  6,
  9,
  7,
  9,
  9,
  6,
  3,
  5,
  9,
  7,
  7,
  5,
  9,
  3,
  7,
  3,
  7,
  9,
  9,
  3,
  8,
  6,
  3,
  5,
  6,
  7,
  3,
  3,
  3,
  8,
  9,
  6,
  9,
  3,
  6,
  9,
  3,
  7,
  5,
  5,
  7,
  3,
  9,
  9,
  3,
  3,
  7,
  8,
  9,
  7,
  8,
  9,
  8,
  3,
  7,
  3,
  3,
  9,
  7,
  3,
  9,
  7,
  3,
  3,
  3,
  7

convertiamo tutto in tensori

devo passare alla fase di embedding perché altrimenti sarebbe una trasformazione ridondante

In [55]:
pip install --upgrade --force-reinstall ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.3/604.3 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.8/387.8 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.7/103.7 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.13
    Uninstalling wcwidth-0.2.13:
      Successfully

In [56]:

#Converting the input data to the tensor , which can be feeded to the model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
test_inputs= torch.tensor(test_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
test_labels=torch.tensor(test_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
test_masks=torch.tensor(test_masks)

In [57]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#Creating the DataLoader which will help us to load data into the GPU/CPU
batch_size = 32

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [58]:
#Loading the pre-trained BERT model from huggingface library

#from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 26,
    output_attentions = False,
    output_hidden_states = False, )

# Teeling the model to run on GPU
#model.cuda()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
import torch
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [60]:
train_dataloader

In [61]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
scheduler

ora dobbiamo definire l'accuracy

In [62]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [63]:
#Creating the helper function to have a watch on elapsed time

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [64]:
# Verifica se CUDA (GPU) è disponibile, altrimenti usa la CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [67]:
torch.cuda.is_available()

True

In [68]:
device

device(type='cuda')

In [69]:
model = model.to(device)

In [70]:
#Let's start the training process

import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels)

        # The call to `model` always returns a tuple, so we need to pull the
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  2,364.    Elapsed: 0:00:09.
  Batch    80  of  2,364.    Elapsed: 0:00:18.
  Batch   120  of  2,364.    Elapsed: 0:00:26.
  Batch   160  of  2,364.    Elapsed: 0:00:35.
  Batch   200  of  2,364.    Elapsed: 0:00:43.
  Batch   240  of  2,364.    Elapsed: 0:00:52.
  Batch   280  of  2,364.    Elapsed: 0:01:01.
  Batch   320  of  2,364.    Elapsed: 0:01:10.
  Batch   360  of  2,364.    Elapsed: 0:01:19.
  Batch   400  of  2,364.    Elapsed: 0:01:28.
  Batch   440  of  2,364.    Elapsed: 0:01:37.
  Batch   480  of  2,364.    Elapsed: 0:01:47.
  Batch   520  of  2,364.    Elapsed: 0:01:56.
  Batch   560  of  2,364.    Elapsed: 0:02:05.
  Batch   600  of  2,364.    Elapsed: 0:02:14.
  Batch   640  of  2,364.    Elapsed: 0:02:23.
  Batch   680  of  2,364.    Elapsed: 0:02:32.
  Batch   720  of  2,364.    Elapsed: 0:02:41.
  Batch   760  of  2,364.    Elapsed: 0:02:50.
  Batch   800  of  2,364.    Elapsed: 0:02:59.
  Batch   840  of

In [71]:
print(loss_values) #Having a view of stored loss values in the list

[0.6565419079865095, 0.5912713757487622, 0.5840233811720978, 0.5767140834641133]


Diminuzione della loss: la perdita di addestramento sta diminuendo progressivamente da 0.65 a 0.57, il che suggerisce che il modello stia imparando a rappresentare meglio i dati.
L'accuratezza sulla validation set è costante a 0.75, il modello non sta migliorando

In [76]:
test_inputs

tensor([[ 101, 1037, 1035,  ...,    0,    0,    0],
        [ 101, 1059, 1035,  ...,    0,    0,    0],
        [ 101, 1037, 1035,  ...,    0,    0,    0],
        ...,
        [ 101, 1037, 1035,  ...,    0,    0,    0],
        [ 101, 1059, 1035,  ...,    0,    0,    0],
        [ 101, 1059, 1035,  ...,    0,    0,    0]])

In [77]:
# Set the batch size.
batch_size = 32

# Create the DataLoader.
prediction_data = TensorDataset(test_inputs, test_masks, test_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [78]:
# Prediction on test set

print('Predicting labels for {:,} test traces...'.format(len(test_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables
predictions , true_labels = [], []

# Predict
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)

  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch

  # Telling the model not to compute or store gradients, saving memory and
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

Predicting labels for 9,453 test traces...


In [86]:
print('Samples: %d' % len(test_labels))


Samples: 9453


In [84]:
test_labels

tensor([8, 9, 3,  ..., 7, 8, 9])

In [87]:
for c in torch.unique(test_labels):
    count_c = (test_labels == c).sum().item()
    print(f'Class {c}: {count_c} samples ({count_c / len(test_labels) * 100:.2f}%)')


Class 2: 35 samples (0.37%)
Class 3: 2432 samples (25.73%)
Class 4: 145 samples (1.53%)
Class 5: 764 samples (8.08%)
Class 6: 1321 samples (13.97%)
Class 7: 1728 samples (18.28%)
Class 8: 886 samples (9.37%)
Class 9: 2096 samples (22.17%)
Class 11: 3 samples (0.03%)
Class 12: 27 samples (0.29%)
Class 20: 2 samples (0.02%)
Class 21: 2 samples (0.02%)
Class 22: 6 samples (0.06%)
Class 23: 3 samples (0.03%)
Class 24: 3 samples (0.03%)


In [88]:
import numpy as np

# Unisci tutte le predizioni e le label in un array singolo
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)


In [91]:
predicted_classes = np.argmax(predictions, axis=1)


In [92]:
predicted_classes

array([8, 9, 3, ..., 7, 8, 9])

In [90]:
true_labels

array([8, 9, 3, ..., 7, 8, 9])

In [93]:
from sklearn.metrics import accuracy_score, classification_report

acc = accuracy_score(true_labels, predicted_classes)
print(f'Accuracy: {acc:.4f}')

print(classification_report(true_labels, predicted_classes))


Accuracy: 0.7477
              precision    recall  f1-score   support

           2       0.88      0.20      0.33        35
           3       0.64      0.80      0.71      2432
           4       0.86      0.99      0.92       145
           5       0.00      0.00      0.00       764
           6       0.98      0.29      0.45      1321
           7       0.61      0.97      0.75      1728
           8       0.88      0.92      0.90       886
           9       0.96      1.00      0.98      2096
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00        27
          20       0.00      0.00      0.00         2
          21       0.00      0.00      0.00         2
          22       0.00      0.00      0.00         6
          23       0.50      0.67      0.57         3
          24       0.00      0.00      0.00         3

    accuracy                           0.75      9453
   macro avg       0.42      0.39      0.37      9453
weighted 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [94]:
true_labels

array([8, 9, 3, ..., 7, 8, 9])

In [95]:
predicted_classes

array([8, 9, 3, ..., 7, 8, 9])

In [ ]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):

  # The predictions for this batch are a 2-column ndarray (one column for "0"
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()

  # Calculate and store the coef for this batch.
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)
  matthews_set.append(matthews)

Calculating Matthews Corr. Coef. for each batch...


In [ ]:

# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('MCC: %.3f' % mcc)

MCC: 0.325


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

# Supponiamo che 'model' sia il tuo modello BERT già addestrato e 'test_dataloader' i dati di test

# Lista per salvare le etichette reali e le previsioni del modello
true_labels = []
predictions = []

# Passo attraverso i dati di test per ottenere le previsioni
model.eval()  # Imposta il modello in modalità di valutazione
for batch in prediction_dataloader:
    b_input_ids, b_input_mask, b_labels = batch
    b_input_ids = b_input_ids.to(device)
    b_input_mask = b_input_mask.to(device)
    b_labels = b_labels.to(device)

    # Fai la previsione senza calcolare i gradienti
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs[0]
    preds = torch.argmax(logits, dim=1)  # Ottieni le previsioni

    # Aggiungi le etichette e le previsioni alle liste
    true_labels.extend(b_labels.cpu().numpy())
    predictions.extend(preds.cpu().numpy())

# Ora calcoliamo le metriche
precision = precision_score(true_labels, predictions, average='weighted')
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')
accuracy = accuracy_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

# Stampa i risultati
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print('Confusion Matrix:')
print(conf_matrix)


Precision: 0.6275
Recall: 0.5352
F1-Score: 0.5118
Accuracy: 0.5352
Confusion Matrix:
[[ 7 13  4]
 [ 0 22  4]
 [ 1 11  9]]


In [ ]:
cm

array([[ 7, 13,  4],
       [ 0, 22,  4],
       [ 1, 11,  9]])